<p align='center'><img src='https://i.hizliresim.com/8a6tv6h.gif'></p>

**GColab**
==
**GColab**, Google Colab'da bulunan Nvidia T4 GPU'nun gücünü kullanarak otomatik olarak görüntü kalitesini artırır. Nvidia T4 GPU'sunu kullanarak **Remini-AI**, görüntü kalitesini Google Colab ortamında üstün ve daha hızlı sonuçlar elde etmek üzere geliştirilmiştir.

In [ ]:
#@title **Paket Kurulumu**

!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

from IPython.display import clear_output
clear_output()
print("Paket kurulumu başarılı")

In [ ]:
#@title **RealESRGAN İle Remini-AI Modeli Oluşturun**
from RealESRGAN import RealESRGAN
from IPython.display import clear_output
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "4" #@param ["2", "4", "8"] {allow-input: false}

try:
    model = RealESRGAN(device, scale=int(model_scale))
    model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')
    clear_output()
    print(f"Model Ölçeği {model_scale} Başarıyla Yüklendi!")
except Exception as e:
    clear_output()
    print('Bir şeyler yanlış gitti..')
    print(e)

In [ ]:
#@title **Yükle ve Sonucu Al**
import os
import requests
from google.colab import files
import shutil
from io import BytesIO
import io
import tarfile
from PIL import Image
from IPython.display import clear_output
import numpy as np

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_telegraph(img_sr):
    # Save the image to a BytesIO buffer
    image_buffer = BytesIO()
    img_sr.save(image_buffer, format='JPEG')
    
    # Upload image to telegra.ph
    files = {'file': ('image.jpg', image_buffer.getvalue())}
    response = requests.post('https://telegra.ph/upload', files=files)
    telegraph_url = response.json()[0]['src']
    
    return telegraph_url

def process_input(filename):
    # Orijinal dosyayı işle
    image = Image.open(filename).convert('RGB')
    sr_image = model.predict(np.array(image))
    
    # Upload processed image to telegra.ph
    telegraph_url = process_telegraph(sr_image)
    print(f'Tamamlandı! Görüntü, {filename} konumuna kaydedildi')
    print(f'Telegraph URL: {telegraph_url}')
    
    # Orijinal dosyayı sil (istenirse)
    # os.remove(filename)

# Dosyaları yükle
try:
    uploaded = files.upload()
    for filename in uploaded.keys():
        print('İşleniyor:', filename)
        process_input(filename)
except KeyboardInterrupt:
    print('Yükleme iptal edildi.')


**Katkıda Bulunanlar✨**
==
**@codermert**
